In [2]:
# Import necessary packages
import stim
import numpy as np
import copy
from typing import List, Tuple

In [1]:
class PauliSummand:
    def __init__(self, phase: complex, pauli: stim.PauliString):
        """ 
        Representation for single element of 'PauliSum'

        Params:
        -------
        phase - Phase of the 'PauliSummand' instance
        pauli - Pauli operator of the 'PauliSummand' instance
        """
        self.phase = phase 
        self.pauli = pauli 
        
        # Store basis paulis for each 'PauliSummand' instance
        self.bases = []

    def get_phase(self):
        """ 
        Get phase of PauliSummand

        Returns:
        --------
        self.phase
        """
        return self.phase 

    def set_phase(self, phase: complex):
        """ 
        Set phase of PauliSummand 

        Params:
        -------
        phase - Phase to set to of 'PauliSummand' instance
        """
        self.phase = phase

    def get_pauli(self):
        """ 
        Get pauli of PauliSummand

        Returns:
        --------
        self.pauli
        """
        return self.pauli

    def set_pauli(self, pauli: stim.PauliString):
        """ 
        Set pauli of PauliSummand 

        Params:
        -------
        pauli - Pauli to set to of 'PauliSummand' instance
        """
        self.pauli = pauli

    def get_bases(self):
        """ 
        Get list of basis elements that generate 'PauliSummand' instance

        Returns:
        --------
        self.bases
        """
        return self.bases 

    def add_to_basis(self, basis_ind: int):
        """ 
        Add basis index corresponding to index of basis element in overall list of 
        Pauli bases

        Params:
        -------
        basis_ind - Index of basis element in overall list of Pauli bases
        """
        self.bases.append(basis_ind)

    def get_pauli_str(self):
        """ 
        Extract just pauli string from stim Pauli representation
        """
        pauli_str = ""
        num_qubits = len(self.pauli)
        for i in range(num_qubits - 1, 0, -1):
            pauli_str = str(self.pauli)[i] + pauli_str
        return pauli_str
            

    def __str__(self):
        pauli_str = ""
        pauli_plus_phase = str(self.pauli)
        pauli_plus_phase_len = len(pauli_plus_phase)
        count = pauli_plus_phase_len - 1
        while (pauli_plus_phase[count] == '_' or pauli_plus_phase[count] == 'X' or 
            pauli_plus_phase[count] == 'Y' or pauli_plus_phase[count] == 'Z'):
            pauli_str = pauli_plus_phase[count] + pauli_str 
            count -= 1
        return "[" + str(self.phase) + " " + pauli_str + "]"

class PauliSum: 
    def __init__(self, *argv):
        """ 
        Representation of contraction of QCSAT instance
        """
        if len(argv) == 0:
            self.sum = []
            self.bases = []
            self.num_bases = 0
        else:
            self.sum = [argv[0]]
            self.bases = [argv[0]]
            self.num_bases = 1

    def get_sum(self):
        """ 
        Get PauliSum in list representation
        """
        return self.sum 

    def get_bases(self):
        """ 
        Get bases for PauliSum
        """
        return self.bases 

    def set_bases(self, basis_list: List):
        """ 
        Set basis for PauliSum

        Params:
        -------
        basis_list - List of bases for PauliSum
        """
        self.bases = basis_list 

    def get_num_bases(self):
        """ 
        Get number of bases in PauliSum
        """
        return self.num_bases

    def set_num_bases(self, basis_count: int):
        """ 
        Set number of basis elements in PauliSum
        """
        self.num_bases = basis_count

    def incr_num_bases(self):
        """ 
        Increment 'num_bases' variable
        """
        self.num_bases += 1

    def __str__(self):
        str_out = ""
        for summand in self.sum:
            str_out += str(summand)
            str_out += " + "
        return str_out[:len(str_out) - 3]
        
    def combine_like_terms(self):
        """ 
        Combine like terms of Pauli Sum
        """
        pass 

    def add_sum_to_sum(self, p):
        """ 
        Add a PauliSum instance to 'self'

        Params:
        -------
        p - PauliSum instance to be added
        """
        self.sum += p.sum 

    def add_summand(self, s: PauliSummand):
        """ 
        Add a PauliSummand to 'self'

        Params:
        -------
        s - PauliSummand instance to be added 
        """
        self.sum.append(s)

    def transform_basis(self, t_gate_loc: int):
        """ 
        Transform basis so that T-gate can be appropriately applied to basis paulis. 
        We want only two bases acting non-trivially at the position where the T-gate 
        is to be applied.

        Params:
        -------
        t_gate_loc - Qubit location where T-gate is being applied
        """
        pauli_dict = {0: '_', 1: 'X', 2: 'Y', 3: 'Z'}
        pauli_column = [pauli_dict[p.__getitem__(t_gate_loc)] for p in self.bases]
        new_basis_paulis = []
        x_count = 0
        x_pos = 0
        y_count = 0
        y_pos = 0
        z_count = 0
        z_pos = 0
        for i in range(len(pauli_column)):
            if (pauli_column[i] == '_'):
                new_basis_paulis.append(self.bases[i])
            if (pauli_column[i] == 'X'):
                if (x_count == 0):
                    new_basis_paulis.append(self.bases[i])
                    x_pos = i
                else:
                    new_basis_paulis.append(self.bases[x_pos] * self.bases[i])
                x_count += 1

            if (pauli_column[i] == 'Y'):
                if (y_count == 0):
                    new_basis_paulis.append(self.bases[i])
                    y_pos = i
                else:
                    new_basis_paulis.append(self.bases[y_pos] * self.bases[i])
                y_count += 1

            if (pauli_column[i] == 'Z'):
                if (z_count == 0):
                    new_basis_paulis.append(self.bases[i])
                    z_pos = i
                else:
                    new_basis_paulis.append(self.bases[z_pos] * self.bases[i])
                z_count += 1 

        if (x_count >= 1 and y_count >= 1):
            new_basis_paulis[x_pos] = new_basis_paulis[x_pos] * new_basis_paulis[y_pos]

        self.bases = new_basis_paulis 
        

    def apply_t_gate(self, num_qubits: int, t_gate_loc: int = 0):
        """ 
        Simulate application of T gate on PauliSum based on which qubit T gate is 
        being applied to 

        Params:
        -------
        num_qubits - Number of qubits
        t_gate_loc - Location where T gate is being applied
        """
        phase_term = 1/np.sqrt(2)
        added = False
        # new_sum = PauliSum()
        X_term = stim.PauliString('_' * t_gate_loc + 'X' + '_' * (num_qubits - t_gate_loc - 1))
        Y_term = stim.PauliString('_' * t_gate_loc + 'Y' + '_' * (num_qubits - t_gate_loc - 1))
        
        # Update Pauli Summands
        for summand in self.sum:
            if (summand.get_pauli().__getitem__(t_gate_loc) == '_' or summand.get_pauli().__getitem__(t_gate_loc) == 'Z'):
                continue
            elif (summand.get_pauli().__getitem__(t_gate_loc) == 'X'):
                new_summand = PauliSummand(phase_term * summand.get_phase(), summand.get_pauli() * X_term * Y_term)
                summand.set_phase(summand.get_phase() * phase_term)
                if (added == False):
                    new_sum = PauliSum(new_summand)
                    added = True 
                else:
                    new_sum.add_summand(new_summand)
            else:
                new_summand = PauliSummand(phase_term * summand.get_phase(), summand.get_pauli() * Y_term * X_term)
                summand.set_phase(-1 * summand.get_phase() * phase_term)
                if (added == False):
                    new_sum = PauliSum(new_summand)
                    added = True 
                else:
                    new_sum.add_summand(new_summand)
        
        # Update Pauli Basis
        

    def apply_tableau(self, t: stim.Tableau):
        """ 
        Apply current stabilizer tableau on PauliSum instance

        Params:
        -------
        t - stabilizer tableau
        """
        for summand in self.sum:
            new_summand = t(summand.get_pauli())
            summand.set_phase(summand.get_phase() * new_summand.sign)
            summand.set_pauli(new_summand)

    def clifford_on_basis(self, cliff: stim.Tableau) -> List:
        """
        Apply clifford operator on all basis_paulis (Change of basis via clifford)

        Params:
        -----------
        cliff - Tableau for clifford operator that is to applied
        """
        self.bases = [cliff(p) for p in self.bases]

    

    

SyntaxError: invalid syntax (3565631884.py, line 237)

In [8]:
for i in range(4,0, -1):
    print(i)

4
3
2
1
